**Character-level text generation with LSTM**
This example demonstrates how to use a LSTM model to generate text character-by-character.

**Setup**


In [1]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

**Prepare the data**

In [2]:
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

600901/600901 [==============================] - 1s 2us/step
Corpus length: 600893
Total chars: 56
Number of sequences: 200285


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


**Build the model: a single LSTM layer**

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

**Prepare the text sampling function**

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

**Train the model**

In [5]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

1565/1565 [==============================] - 16s 6ms/step - loss: 1.9145

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "l does not operate on will wherever "eff"
...Generated:  ect and to a suffering and and and and and the serse and to the self the self and to the self with the self-consequent to a present of the self and to a sorting and the sense of a conscience and to a proffeence of his self-conscience of the are and to a sist and the sense of a perscemple of the sense of the order to a something and the self-consequent of a conscience of the probles and a propinity

...Diversity: 0.5
...Generating with seed: "l does not operate on will wherever "eff"
...Generated:  ect and more that on this mither to be ser of all the to who are to on the sense of has to in the proble man and have the presing and reterness to he has the sabrittion of a self and moralive, and the has and to a serfly and remains to a strengte, the supprears of a reasing and polse to his

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...Generated:  eaks as really the religion the most good or an extent the soul of the end and experience the translated nowadays, some samely that even a standard the same man and instinctive, as the person as it in order to the conscience for the general the life, the belief in the philosopher is not believe and for the laws of the sense of indeness of the contrad the connectant, as it is and eternal the presen

...Diversity: 1.0
...Generating with seed: "lish, with their profound mediocrity, br"
...Generated:  ee some the cuonrovering espitan to en prijfnemgness in that also. what: estably are man is belong naturaking whateve down. his heart, when i speply.   9a by kind is too fless ofted, incorpr which as almost self, brake of geverfulnes of the "forises," who experience and however above ourselves, seberation.=--it is precime distance, whose words, as just, even which as only the freisciment of an in 

...Diversity: 1.2
...Generating with seed: "lish, with their profound mediocrity